# Generating Word invoices from a CSV file

In [1]:
from docx import Document
import pandas as pd
from pathlib import Path

# Configurable tax rate
TAX_RATE = 10  # in percent

# Load CSV data
df = pd.read_csv("invoice_data.csv")

# Group by invoice number
invoices = df.groupby("Invoice Number")

for invoice_number, items in invoices:
    print(f"Invoice #: {invoice_number}")
    display(items)

Invoice #: INV001


,Invoice Number,Date,Client Name,Item,Quantity,Unit Price
0,INV001,2025-04-25,Acme Corp,Widget A,2,25.0
1,INV001,2025-04-25,Acme Corp,Widget B,1,40.0


Invoice #: INV002


,Invoice Number,Date,Client Name,Item,Quantity,Unit Price
2,INV002,2025-04-26,Beta LLC,Gadget A,5,15.0
3,INV002,2025-04-26,Beta LLC,Gadget B,2,30.0


### Helper functions

In [2]:
# Helper to format currency
def format_currency(amount):
    return f"${amount:.2f}"

format_currency(43.6351) # should output '$43.64'

'$43.64'

In [3]:
# Load the template
template_path = "invoice_template.docx"
output_dir = Path("output")
output_dir.mkdir(exist_ok=True)

for invoice_number, items in invoices:
    doc = Document(template_path)

    # Extract common fields
    client_name = items["Client Name"].iloc[0]
    date = items["Date"].iloc[0]

    # Calculate totals
    items["Total"] = items["Quantity"] * items["Unit Price"]
    subtotal = items["Total"].sum()
    tax = subtotal * TAX_RATE / 100
    total = subtotal + tax

    # Replace placeholders
    for p in doc.paragraphs:
        if "{{invoice_number}}" in p.text:
            p.text = f"Invoice Number: {invoice_number}"
        elif "{{date}}" in p.text:
            p.text = f"Date: {date}"
        elif "{{client_name}}" in p.text:
            p.text = f"Client: {client_name}"
        elif "{{subtotal}}" in p.text:
            p.text = f"Subtotal: {format_currency(subtotal)}"
        elif "{{tax_rate}}" in p.text:
            p.text = f"{TAX_RATE}"
        elif "{{tax}}" in p.text:
            p.text = f"Tax: {format_currency(tax)}"
        elif "{{total}}" in p.text:
            p.text = f"Total: {format_currency(total)}"

    # Insert items into the table
    for i, p in enumerate(doc.paragraphs):
        if "{{items_table}}" in p.text:
            # Remove placeholder paragraph
            parent = p._element.getparent()
            parent.remove(p._element)
            # Get the first table
            table = doc.tables[0]
            for _, row in items.iterrows():
                new_row = table.add_row().cells
                new_row[0].text = row["Item"]
                new_row[1].text = str(row["Quantity"])
                new_row[2].text = format_currency(row["Unit Price"])
                new_row[3].text = format_currency(row["Total"])
            break

    # Save the document
    output_path = output_dir / f"{invoice_number}.docx"
    doc.save(output_path)

print(f"Invoices saved to: {output_dir.resolve()}")

Invoices saved to: /home/jupyter-subwaymatch/mba564b-2025-redesign/generate-invoices-exercise/output
